In [ ]:
pip install matplotlib --user

In [ ]:
import os 
import io
from transformers.models.imagegpt.modeling_imagegpt import IMAGEGPT_INPUTS_DOCSTRING
from transformers import GPT2Tokenizer
import pandas as pd 
from pandas.io import json
from numpy import nan
import time
import csv
import shutil
import datetime
import pytz
import re
import matplotlib.pyplot as plt
import numpy as np
import time
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from huggingface_hub import login
import transformers
print(transformers.__version__)

In [ ]:
setting = "without_Substances"
shot = "0shot"
model_name = "google/gemma-2-9b-it"
model_engine = "google-gemma-2-9b-it_llama_prompt2_"+ setting +"_"+ shot + "/"
temperature =  0.0001
max_token = 100

In [ ]:
HF_TOKEN = "your_token"
login(token = HF_TOKEN)

In [ ]:
################## User input in txt file #####################
def user_input():
    file_path = "Prompts/Prompt_VAC/"+ setting +"_" + shot + "_Single_Abstract.txt"

    with open(file_path, 'r') as file:
        user_input = file.read()
    dataset = "VAC_Cancer_Vaccine_adjuvant"
    query = user_input

    print(dataset)
    print(temperature)
    print(query)
    
    return dataset, temperature,  query

dataset, temperature, query = user_input()


#Single_Abstract_preprocessed_with_Substances_only_with_substances_merged_columns
if dataset == "VAC_Cancer_Vaccine_adjuvant":
    Implementation_base_path_input = 'Dataset/'+dataset+'/Single_Abstract_preprocessed_with_Substances_only_' + setting + '_merged_columns'

print(Implementation_base_path_input)

In [ ]:
# Corrected code with max_memory in the right place
n_gpus = torch.cuda.device_count()
print("N GPUS: ", n_gpus)
max_memory = f'{40960}MB'

text_generator = pipeline(
    "text-generation",
    model=model_name,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "max_memory": {i: max_memory for i in range(n_gpus)}  # Moved here
    },
    device_map="auto",
)

In [ ]:
def get_response_LLAMA(prompt):
  
  messages = [
      {"role": "user", "content": prompt},
  ]
  sequences = text_generator(messages, max_new_tokens=max_token)
  gen_text = sequences[0]["generated_text"][-1]["content"]
  return gen_text

In [ ]:
#count_input_tokens(Implementation_base_path_input)

from transformers import GPT2Tokenizer
from math import ceil

folder_path = Implementation_base_path_input


def count_input_tokens_no_use(folder_path):
  print(folder_path)
  # Initialize GPT2 tokenizer
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  # Get list of file names in folder
  file_names = os.listdir(folder_path)
  # List to store file names with more than 1500 tokens
  large_token_files = []
  x = 0
  y = 0
  parts = 0
  # Loop through each file and calculate number of tokens
  for file_name in file_names:
      # Check if file is a text file
      if file_name.endswith(".txt"):
          # Read file contents
          with open(os.path.join(folder_path, file_name), "r" , encoding='utf-8') as f:
              file_contents = f.read()
          with open(os.path.join(folder_path, file_name), "r" , encoding='utf-8') as fp:
              num_input_line = len(fp.readlines())
          # Calculate number of tokens
          num_tokens = len(tokenizer.encode(file_contents))
          num_output_tokens  = num_input_line*100
          total_token = num_tokens + num_output_tokens
          if total_token > 16000:
            print(f"{file_name}: {total_token} tokens \t parts needed: {ceil(num_output_tokens/4000)}")
            x= x+1
            parts = parts + ceil(num_output_tokens/4000)
            large_token_files.append(file_name)
          else:
            y = y+1
            parts = parts +1
        
  print(x)
  print(y)
  print(parts)
  return large_token_files
              
            
large_token_files = count_input_tokens_no_use(folder_path)
print(large_token_files)


In [ ]:
import os

def count_txt_files_in_folder(folder_path):
    txt_file_count = 0
    # os.walk() generates the file names in a directory tree by walking either top-down or bottom-up.
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                txt_file_count += 1  # Increment the count if the file ends with .txt
    return txt_file_count

def count_txt_files_excluding_checkpoints(folder_path):
    txt_file_count = 0
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt') and '-checkpoint' not in file:
                txt_file_count += 1
    return txt_file_count


# Example usage
folder_path_original = Implementation_base_path_input

total_txt_files_original = count_txt_files_excluding_checkpoints(folder_path_original)
print(f"Total number of original .txt files: {total_txt_files_original}")




In [ ]:
# 10 fold
def call_LLM(Implementation_base_path_input, Implementation_base_path_output, Run, temperature, query):
    print("Run no: ", Run+1)
    #print("Temperature: ", temperature)
    #print("Query:", query)
    #print("Dataset: ", dataset)
    r = 291
    for fold_no in range(1, r):
        with open(os.path.join(Implementation_base_path_input, f'Abstract_{fold_no}.txt')) as f:
            Sentences = f.read()

        prompt = f"""
        {query}
        {Sentences}
        ### Task Output:"""

        print(prompt)
            
        message = get_response_LLAMA(prompt)

        last_line = message.strip().split('\n')[-1]
        if "Done" in last_line:
            print(f"Abstract no= {fold_no}, Run = {Run}, Temperature = {temperature}, Complete")
        else:
            print(f"Abstract no= {fold_no}, Run = {Run}, Temperature = {temperature}, Possibly Incomplete")

        with open(
            os.path.join(Implementation_base_path_output + str(Run + 1) + f'_Abstract_{fold_no}.txt'), "a"
        ) as f:
            print(message, file=f)


In [ ]:
import os

def output_path(Run_no, dataset, temperature):
    base = "Output/"+model_engine+"/Single_Abstract_With_Substances_Only_" + dataset + "_T"+ str(temperature)+ '_16December/'   ######################## without
    extension_path = dataset + "_T" + str(temperature) + "_Run" + str(Run_no) + '/'
    Implementation_base_path_output = os.path.join(base, extension_path)

    # Use exist_ok=True to avoid FileExistsError in a multiprocessing context
    os.makedirs(Implementation_base_path_output, exist_ok=True)
        
    return Implementation_base_path_output


In [ ]:
input_file_names = []

for file_name in os.listdir(Implementation_base_path_input):
    
    if os.path.isfile(os.path.join(Implementation_base_path_input, file_name)):
        input_file_names.append(file_name)

file_count = len(input_file_names)

print("Number of files in the folder:", file_count)

In [ ]:
for Run in range(3):
    R = Run+1
    Implementation_base_path_output = output_path(R, dataset, temperature)
    print("Implementation base path input:", Implementation_base_path_input)
    print("Implementation base path output:", Implementation_base_path_output)
    call_LLM(Implementation_base_path_input, Implementation_base_path_output, Run, temperature, query)

In [ ]:
print("Done")